<a href="https://colab.research.google.com/github/Anand0111/PotHoleDetection/blob/main/PotHoleDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import pygame
pygame.init()
pygame.mixer.music.load('buzzer.wav')
import smtplib
import pygame
from matplotlib import pyplot as plt
# Image Processing Techniques such as Contour, Canny Edge detection, Thresholding, Blurring is applied in the below function.
def sketch_transform(image):
    
    image_grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image_grayscale_blurred = cv2.GaussianBlur(image_grayscale, (7,7), 0)
    image_canny = cv2.Canny(image_grayscale_blurred, 1, 220)
    _, mask = image_canny_inverted = cv2.threshold(image_canny,250, 255, cv2.THRESH_BINARY_INV)
    contours =  cv2.findContours(mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)[-2]
    for c in contours:
        area=cv2.contourArea(c)
        if area>150:
            cv2.drawContours(mask, [c], -1, (0,0,255), 1)
            rect = cv2.boundingRect(c)
            if rect[2] < 20 or rect[3] < 20: 
                continue
           
            x,y,w,h = rect
            cv2.rectangle(mask,(x,y),(x+w,y+h),(0,255,0),3)
         
            cv2.putText(mask,'ALERT!!!',(x+w+40,y+h),1,2.0,(0,0,255))
           
            pygame.mixer.music.play()
            pygame.time.wait(100)

            plt.title("Moth Detected Pothole Image")

    return mask

# The video input is received and processed.
cam_capture = cv2.VideoCapture("videoplayback.mp4")

fcc=cv2.VideoWriter_fourcc(*'XVID')
out=cv2.VideoWriter('test.mp4',fcc,5,(850,720))


upper_left = (50,50)
bottom_right = (350,400)
while True:
    _, image_frame = cam_capture.read()
    image_frame=cv2.resize(image_frame,(500,500),fx=0,fy=0,interpolation=cv2.INTER_CUBIC)
    
    r = cv2.rectangle(image_frame, upper_left, bottom_right, (0, 0,255), 8)
    rect_img = image_frame[upper_left[1] : bottom_right[1], upper_left[0] : bottom_right[0]]
    
    sketcher_rect = rect_img
    sketcher_rect = sketch_transform(sketcher_rect)


    sketcher_rect_rgb = cv2.cvtColor(sketcher_rect, cv2.COLOR_GRAY2RGB)
    
    
    image_frame[upper_left[1] : bottom_right[1], upper_left[0] : bottom_right[0]] = sketcher_rect_rgb
    cv2.imshow("Sketcher ROI", image_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cam_capture.release()
cv2.destroyAllWindows()

In [ ]:
import smtplib

server = smtplib.SMTP_SSL("smtp.gmail.com", 465)
server.login("youremail@gmail.com", "password")
server.sendmail("youremail@gmail.com",
                "sendermail@gmail.com",
                "Subject:ALERT!!!\n\n"
                "Pothole Detected\n\n"
                "This is an information regarding a pothole that was detected by a user.For more information please contact the user via the same mail address.\n"
                "Make sure to address this public concern.\n\n"
                "Thank You")
server.quit()
#Email addresses and passwords are deliberately removed.
#Instead of youremail@gmail.com place Your email address, same is true for the sender's side as well.